In [1]:
import os
os.getcwd()

'/'

In [2]:
! ls -alh /var/model

total 63G
drwxrwxr-x 5 1001 1001 4.0K Nov 13 19:09 .
drwxr-xr-x 1 root root 4.0K Nov 14 15:49 ..
-rw-r--r-- 1 1001 1001 6.1K Nov 13 19:09 .DS_Store
-rw-r--r-- 1 1001 1001 4.0K Nov 13 19:09 ._.DS_Store
drwxr-xr-x 2 root root 4.0K Nov 11 02:21 .ipynb_checkpoints
-rw-rw-r-- 1 1001 1001 3.7K Nov  6 17:14 README.md
drwxr-xr-x 2 root root 4.0K Nov 11 02:56 checkpoints
-rw-rw-r-- 1 1001 1001  638 Nov  6 17:14 config.json
-rw-rw-r-- 1 1001 1001  116 Nov  6 17:14 generation_config.json
-rw-r--r-- 1 1001 1001  984 Nov  7 18:12 model.bak
-rw-r--r-- 1 1001 1001  984 Nov  6 21:02 model.yaml
drwxr-xr-x 3 root root 4.0K Nov 11 03:06 output
-rw-rw-r-- 1 1001 1001 9.2G Nov  6 17:28 pytorch_model-00001-of-00007.bin
-rw-rw-r-- 1 1001 1001 9.1G Nov  6 17:28 pytorch_model-00002-of-00007.bin
-rw-rw-r-- 1 1001 1001 9.1G Nov  6 17:28 pytorch_model-00003-of-00007.bin
-rw-rw-r-- 1 1001 1001 9.1G Nov  6 17:28 pytorch_model-00004-of-00007.bin
-rw-rw-r-- 1 1001 1001 9.1G Nov  6 17:28 pytorch_model-00005-of-00007.b

In [3]:
os.chdir( "/var/model" )

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained( "." )
model     = AutoModelForCausalLM.from_pretrained( ".", quantization_config=bnb_config, low_cpu_mem_usage=True, device_map="auto", low_cpu_mem_usage=Tru ) # , local_files_only=True

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training( model )
type( model )

transformers.models.llama.modeling_llama.LlamaForCausalLM

In [7]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 8192)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=8192, out_features=8192, bias=False)
          (k_proj): Linear4bit(in_features=8192, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=8192, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=8192, out_features=8192, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=8192, out_features=22016, bias=False)
          (up_proj): Linear4bit(in_features=8192, out_features=22016, bias=False)
          (down_proj): Linear4bit(in_features=22016, out_features=8192, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRM

In [1]:
def print_trainable_parameters( model ):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [10]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, 
    lora_alpha=32, 
    target_modules=[ "q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj" ], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 108920832 || all params: 17243447296 || trainable%: 0.6316650616913858


In [9]:
# from datasets import load_dataset
# 
# data = load_dataset("Abirate/english_quotes")
# data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

transformers.models.llama.modeling_llama.LlamaForCausalLM

In [19]:
# from datasets import load_dataset
# 
# yelp_dataset = load_dataset( "yelp_review_full" )
# 
# yelp_dataset[ "train" ][ 100 ]

In [20]:
# def tokenize_function( examples ):
#     return tokenizer( examples[ "text" ], padding="max_length", truncation=True )
# 
# 
# tokenized_yelp_datasets = yelp_dataset.map( tokenize_function, batched=True )
# 
# small_yelp_train_dataset = tokenized_yelp_datasets[ "train" ].shuffle( seed=42 ).select( range( 5000 ) )
# small_yelp_test_dataset  = tokenized_yelp_datasets[ "test" ].shuffle( seed=42 ).select( range( 5000 ) )


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [15]:
#Pass in a prompt and infer with the model
prompt = 'Q: Create a detailed description for the following product: Corelogic Smooth Mouse, belonging to category: Optical Mouse\nA:'
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1553: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


four score and seven years ago our fathers brought forth on this continent a new nation, conceived in liberty, and dedicated to the proposition that all men are created equal.

Now we are engaged in a great civil war, testing whether that nation, or any nation so conceived and so dedicated, can long endure. We are met on a great battlefield of that war. We have come to dedicate a portion of that field, as a final resting place for those who here gave their lives that that nation might live. It is altogether fitting and proper that we should do this.

But, in a larger sense, we can not dedicate, we can not consecrate, we can not hallow this ground. The brave men, living and dead, who struggled here, have consecrated it, far above our poor power to add or detract. The world will little note, nor long remember what we say here, but it can never forget what they did here. It
CPU times: user 1min 12s, sys: 40.9 s, total: 1min 52s
Wall time: 1min 52s


In [14]:
tokenizer = AutoTokenizer.from_pretrained( ".", use_fast=True )
text_generator_4bit = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PhiForCa

In [14]:
import transformers

num_training_steps = 20
# progress_bar = tqdm(range(num_training_steps))

# this is a fairly common redefinition of the padding token
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=num_training_steps,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="checkpoints",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

## using trl SFTTrainer

In [11]:
model.save_pretrained( "output" )

In [14]:
%%time
text = "four score and seven years ago"
device = "cuda:1"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1553: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


four score and seven years ago our fathers brought forth on this continent a new nation, conceived in liberty, and dedicated to the proposition that all men are created equal.

Now we are engaged in a great civil war, testing whether that nation, or any nation so conceived and so dedicated, can long endure. We are met on a great battlefield of that war. We have come to dedicate a portion of that field, as a final resting place for those who here gave their lives that that nation might live. It is altogether fitting and proper that we should do this.

But, in a larger sense, we can not dedicate, we can not consecrate, we can not hallow this ground. The brave men, living and dead, who struggled here, have consecrated it, far above our poor power to add or detract. The world will little note, nor long remember what we say here, but it can never forget what they did here. It
CPU times: user 1min 11s, sys: 40.1 s, total: 1min 51s
Wall time: 1min 51s
